In [1]:
import pydub
import librosa
import mir_eval

In [2]:
from pydub import AudioSegment
from scipy.io.wavfile import read

+ wjs0和TIMIT数据集的音频的文件头都是NIST，这种文件称为SPH文件，它们都不是正确格式的wav文件，因此借助开源的SPHFile库先转换为wav文件

In [3]:
from sphfile import SPHFile

In [28]:
sph = SPHFile('./timit_examples/SI1418.WAV')

+ 打印sph文件的格式

In [29]:
print(sph.format)

{'sample_rate': 16000, 'sample_sig_bits': 16, 'sample_n_bytes': 2, 'sample_byte_format': '01', 'channel_count': 1, 'sample_coding': 'pcm'}


+ 将sph文件转换为wav格式

In [30]:
sph.write_wav('./timit_examples/SI1418.wav')

'./timit_examples/SI1418.wav'

+ pydub的AudioSegment只能读取正确格式的wav文件，因此需要上述SPHFile库先预先转化

In [34]:
audio = AudioSegment.from_wav('./timit_examples/SI1418.wav')

+ 更加广泛的用法,from_file

In [53]:
audio_ = AudioSegment.from_file('./timit_examples/SI1418.wav', format='wav', frame_rate=16000, channels=1)
audio_.frame_rate, audio_.channels

(16000, 1)

+ sample_width只得是一个样本点占用了多少个位，1代表占用8位，2代表占用8位（CD格式的，采样频率为44100Hz，另外DVD为48000Hz），4代表占用32位，而frame_width是要考虑通道的，frame_width = sample_width * num_channels

In [56]:
print(audio.frame_width, audio.sample_width)

2 2


+ 以1000ms（1s）为单位切割出若干音频片段

In [35]:
for segment in audio[::1000]:
    print(len(segment))
print(len(audio))

1000
1000
1000
1000
1000
574
5574


In [36]:
audio

+ dBFS (db relative to the maximum possible loudness)

In [24]:
print(audio.dBFS)

-35.31427238781313 562


+ 于duration以毫秒计算，5574ms就是5.5744375s(duration_seconds计算)，采样频率为16000Hz，也就是每秒采样16000个点(samples/frames，一般情况下mono音频的frame的数目等于sample的数目，双声道下frame=2\*sample)，在做stft的时候，frame可能也会包含若干连续的sample，总之sample才是一个样本点)，那么frame的数目为5.5744375 \* 16000 = 89191，参数ms指定计算x ms内有多少个frame <br>
<img src='timit_examples/frame.png'>

In [57]:
print(audio.frame_count(), audio.frame_count(ms=100), audio.frame_rate, audio.duration_seconds)

89191.0 1600.0 16000 5.5744375


In [31]:
sph = SPHFile('./timit_examples/SI788.WAV')
sph.write_wav('./timit_examples/SI788.wav')

'./timit_examples/SI788.wav'

In [32]:
audio2 = AudioSegment.from_wav('./timit_examples/SI788.wav')

In [33]:
audio2

+ ovelay的作用是叠加，也叫mix，确保了两个audio混在一起后能同步播放，如果audio的长度小于audio2的长度，那么audio2的长度会被截断，ovelay的作用与拼接append是不一样的，audio + audio2是指将两个audio的接到一起，按顺序播放，而不是同步，另外，audio +/- 6指的是提高或者下降loudness

In [37]:
combined = audio.overlay(audio2)

In [38]:
combined

+ 将mix后的audio写入一个wav文件

In [39]:
combined.export('./timit_examples/combined.wav', format='wav')

<_io.BufferedRandom name='./timit_examples/combined.wav'>

In [40]:
combined_read = AudioSegment.from_wav('./timit_examples/combined.wav')

In [41]:
combined_read